In [9]:
from src.partition import Partition
from src.generators import PartitionClass
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

from keras.models import Sequential, Model
from keras.layers import (
    Dense, Conv2D, Flatten, 
    ConvLSTM2D, BatchNormalization,
    Conv3D, Input, MaxPooling2D, UpSampling2D,
)

from keras.callbacks.callbacks import EarlyStopping, ReduceLROnPlateau

Using TensorFlow backend.


# Training Data

In [ ]:
n = 50

training_partitions = [p for p in PartitionClass(n).grid_group(n)]
training_matrices = [p.fit_matrix(n) for p in training_partitions]
training_targets = [p._next_oblak_step[1].fit_matrix(n) for p in training_partitions]

X = np.array(training_matrices)
y = np.array(training_targets)

X_train, X_test, y_train, y_test = train_test_split(X, y)

X_train = X_train.reshape(len(X_train), n, n, 1)
X_test = X_test.reshape(len(X_test), n, n, 1)
y_train = y_train.reshape(len(y_train), n, n, 1)
y_test = y_test.reshape(len(y_test), n, n, 1)

# Keras Stability Model

In [7]:
n = 50

training_partitions = [p for p in PartitionClass(n).grid_group(n)]
training_targets = [p.is_stable for p in training_partitions]

X = np.array(training_matrices)
y = np.array(training_targets)

label_binarizer = LabelBinarizer()
y = label_binarizer.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y)

X_train = X_train.reshape(len(X_train), n, n, 1)
X_test = X_test.reshape(len(X_test), n, n, 1)
# y_train = y_train.reshape(len(y_train), n, n, 1)
# y_test = y_test.reshape(len(y_test), n, n, 1)

In [8]:
#create model
stability_model = Sequential()
#add model layers
stability_model.add(Conv2D(64, kernel_size=3, activation="relu", input_shape=(n, n, 1)))
stability_model.add(MaxPooling2D((2, 2), padding='same'))
stability_model.add(Conv2D(32, kernel_size=2, activation="relu"))
stability_model.add(MaxPooling2D((2, 2), padding='same'))
stability_model.add(Conv2D(16, kernel_size=2, activation="relu"))
stability_model.add(MaxPooling2D((2, 2), padding='same'))
stability_model.add(Flatten())
stability_model.add(Dense(units=500, activation='relu'))
stability_model.add(Dense(units=100, activation='relu'))
stability_model.add(Dense(units=20, activation='relu'))
stability_model.add(Dense(units=len(y[0]), activation='sigmoid'))

In [9]:
stability_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
es = EarlyStopping(monitor='val_loss', min_delta=.001, patience=5, verbose=0, mode='auto', baseline=None, restore_best_weights=False)
cbs = [es, lr]

In [15]:
stability_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=1000, callbacks=cbs, batch_size=1000, workers = 4)

Train on 971977 samples, validate on 323993 samples
Epoch 1/1000
971000/971977 [============================>.] - ETA: 1s - loss: 1.3616e-05 - accuracy: 1.0000

KeyboardInterrupt: 

In [17]:
# 100% accuracy
print(f'Accuracy: {stability_model.evaluate(X_test, y_test, verbose = False)[1] * 100}%')
print(f'AUC Score: {roc_auc_score(y_test, stability_model.predict_proba(X_test)) * 100}%')

Accuracy: 100.0%
AUC Score: 100.0%


In [19]:
test=  stability_model.predict_proba(X_test)

In [23]:
import pandas as pd
test = pd.Series(test.flatten())


In [26]:
test[test < .5].describe()

count    3.214310e+05
mean     3.412734e-07
std      2.379810e-05
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      8.120298e-03
dtype: float64

In [27]:
for layer in stability_model.layers:
    layer.trainable = False
    
stability_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
stability_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 64)        640       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 23, 23, 32)        8224      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 16)        2064      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 6, 6, 16)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)              

## Keras Model

In [118]:
input_img = Input(shape=(28, 28, 1))  # adapt this if using `channels_first` image data format

x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (4, 4, 8) i.e. 128-dimensional

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

model = Model(input_img, decoded)

In [120]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 14, 14, 8)         1160      
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 7, 7, 8)           0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 7, 7, 8)           584       
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 4, 4, 8)           0   

In [25]:
model.compile(optimizer='adadelta', loss='binary_crossentropy', metrics = ['accuracy'])

# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [26]:
lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
es = EarlyStopping(monitor='val_loss', min_delta=.0001, patience=10, verbose=0, mode='auto', baseline=None, restore_best_weights=False)
cbs = [es, lr]

In [284]:
from itertools import chain
def grid_group(self, n):
    assert n <= self.n, "The provided n must be less than or equal to the n of the partition class."

    return chain(self.generate_partitions(n = i) for i in range(n))

In [31]:
model.fit(X_train, y_train, validation_data=(X_test, y_test),
          epochs=1000, callbacks=cbs, batch_size=100, workers = 4)

ValueError: `class_weight` not supported for 3+ dimensional targets.

## Validation

In [44]:
Partition([int(sum(x)) for x in prediction.round()[0].reshape(n, n) if sum(x)])

(2, 1, 1)

In [24]:
for partition in list(PartitionClass(n).partitions)[-20:]:
    model_input = np.array(partition.matrix).reshape(-1, *(n, n, 1))
    prediction =  model.predict(model_input)
    pred_partition = Partition([int(sum(x)) for x in prediction.round()[0].reshape(n, n) if sum(x)])
    
    print(partition)
    print()
    print(partition._next_oblak_step[1])
    print()
    print(pred_partition)
    print("NEXT")


ValueError: cannot reshape array of size 1 into shape (28,28)

In [30]:
print()

['▢▢▢▢', '▢▢▢', '▢▢']

In [27]:
stables = [p for p in PartitionClass(13).filter_by_attribute('is_stable')]
unstables = [p for p in PartitionClass(20).filter_by_attribute('is_stable', False)]

In [58]:
input_array = np.zeros((n, n))

p = np.random.choice(stables)

for ind, part in enumerate(p):
    input_array[ind][:part] = 1
    
input_array = input_array.reshape(-1, *(n, n, 1))

print(p)
print(label_binarizer.inverse_transform(stability_model.predict(input_array))[0])
print(stability_model.predict(input_array)[0])

▢▢▢▢▢▢▢▢▢▢▢▢
▢
True
[1.]


In [117]:
[{"True": 1, "False": 0}.get(x, x) for x in ['False', 1, 'oabf']]

[0, 1, 'oabf']